## After I scraped Politico, it turned out their individual political parties were rife with error. So I scraped them from the New York Times and merged them back. As an added bonus, NYT had a bunch of races like state legislatures that Politico didn't have. As another bonus, all the data for New York Times election results is hardcoded as a Javascript object in the page source!

In [1]:
from bs4 import BeautifulSoup
import re
import json
import requests
import os

In [2]:
# first use Selenium to get a dict of urls for state results.
if not os.path.isfile('nyt_state2url.json'):
    from selenium import webdriver
    from selenium.webdriver.support.ui import Select
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(10)
    driver.get("http://www.nytimes.com/elections/results/alabama")
    # click on 'State Results' item to get list of states and URLs
    toexpand = driver.find_element_by_xpath('//*[@id="shell"]/nav/div/a[6]')
    toexpand.click()
    outerhtml = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(outerhtml, 'lxml')
    nav = soup.find_all('nav', attrs={'class': "eln-navigation-states"})[0]
    state2url = {}
    for tag in nav.find_all('a'):
        state2url[tag.attrs['title']] = tag.attrs['href']
    driver.quit()
    with open('nyt_state2url.json', 'w+') as f:
        f.write(json.dumps(state2url))
else:
    with open('nyt_state2url.json', 'r') as f:
        state2url = json.load(f)
            

In [5]:
nytresults = {}

In [6]:
for state, url in state2url.items():
    print(state)
    r = requests.get(url)
    assert r.status_code == 200
    soup = BeautifulSoup(r.text, 'lxml')
    for script in soup.find_all('script'):
        if script.text.find('eln_election_date') != -1:
            break
    for line in script.text.split('\n'):
        if line.find('eln_races = [{"') != -1:
            break
    blob = re.search("eln_races\s+=\s+(.+),", line).group(1)
    blob = re.sub('<a href="(.+?)">', r'a href=\"\1\">', blob) # escape the double quotes inside a url inside json
    nytresults[state] = json.loads(blob)
    

Missouri
South Dakota
Ohio
Colorado
Nevada
North Carolina
Texas
Virginia
Florida
Arkansas
New Mexico
District of Columbia
Tennessee
Oklahoma
North Dakota
Wyoming
Iowa
New Jersey
Minnesota
Rhode Island
Delaware
Wisconsin
Mississippi
Michigan
Indiana
Alaska
California
Massachusetts
New Hampshire
Connecticut
Louisiana
West Virginia
Georgia
Maine
Illinois
Kansas
Arizona
New York
South Carolina
Idaho
Pennsylvania
Montana
Kentucky
Alabama
Utah
Vermont
Nebraska
Oregon
Maryland
Hawaii
Washington


In [7]:
with open('../nyt_election_2016_by_state.json', 'w+') as f:
    f.write(json.dumps(nytresults))

# Part 2: Extracting data

In [1]:
import json
import pandas as pd

with open('../nyt_election_2016_by_state.json', 'r') as f:
    nytresults = json.load(f)

In [ ]:
byoffice = {}

In [11]:
for state, results in nytresults.items():
    for result in results:
        office = result['office']
        if office not in byoffice.keys():
            byoffice[office] = {}
        else:
            byoffice[office][state] = {k: v for k,v in result.items() if }
        print(result['office'])
    break

President
U.S. House
U.S. House
U.S. House
U.S. House
U.S. House
U.S. House
U.S. House
U.S. House
U.S. House
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State Senate
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House
State House

In [4]:
z = z[0]

In [5]:
type(z)

dict

In [8]:
import pprint
pprint.pprint(z)

{'article_url': '',
 'candidates': [{'candidate_id': 'trump-8639',
                 'candidate_key': 'trumpd',
                 'electoral_votes': 9,
                 'first_name': 'Donald',
                 'last_name': 'Trump',
                 'name_display': 'Donald J. Trump',
                 'order': 2,
                 'party_id': 'republican',
                 'percent': 62.9,
                 'percent_display': '62.9',
                 'votes': 1306925,
                 'winner': True},
                {'candidate_id': 'clinton-1746',
                 'candidate_key': 'clintonh',
                 'electoral_votes': 0,
                 'first_name': 'Hillary',
                 'last_name': 'Clinton',
                 'name_display': 'Hillary Clinton',
                 'order': 1,
                 'party_id': 'democrat',
                 'percent': 34.6,
                 'percent_display': '34.6',
                 'votes': 718084},
                {'candidate_id': 'johnson-31708

In [ ]:
['preview_text', 'office', 'guid', 'nyt_rating', 'path', 'poll_display', 'poll_time', 
'race_best_sentence', 'race_best_sentence_is_fresh', 'race_headline', 'race_id', 'race_name',
'race_slug', 'race_type', 'report']